# Text Processing

## Capturing Text Data

### Plain Text

In [37]:
import json
import os
import re
import requests

from bs4 import BeautifulSoup
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

In [5]:
!ls ../..

CODEOWNERS               data                     requirements.txt
README.md                mine                     sentiment_analysis.ipynb
cache                    nltk_data                text_processing.ipynb


In [30]:
DATA = '../../data'
NLTK_DATA = '../../nltk_data'

In [8]:
# Read in a plain text file
with open(os.path.join(DATA, 'hieroglyph.txt'), 'r') as f:
    text = f.read()
    print(text)

Hieroglyphic writing dates from c. 3000 BC, and is composed of hundreds of symbols. A hieroglyph can represent a word, a sound, or a silent determinative; and the same symbol can serve different purposes in different contexts. Hieroglyphs were a formal script, used on stone monuments and in tombs, that could be as detailed as individual works of art.



### Tabular Data

In [10]:
# Extract text column from a dataframe
df = pd.read_csv(os.path.join(DATA, 'news.csv'))
df.head()[['publisher', 'title']]

# Convert text column to lowercase
df['title'] = df['title'].str.lower()
df.head()[['publisher', 'title']]

,publisher,title
0,Livemint,fed's charles plosser sees high bar for change...
1,IFA Magazine,us open: stocks fall after fed official hints ...
2,IFA Magazine,"fed risks falling 'behind the curve', charles ..."
3,Moneynews,fed's plosser: nasty weather has curbed job gr...
4,NASDAQ,plosser: fed may have to accelerate tapering pace


### Online Resource

In [11]:
# Fetch data from a REST API
r = requests.get('https://quotes.rest/qod.json')
res = r.json()
print(json.dumps(res, indent=4))

# Extract relevant object and field
q = res['contents']['quotes'][0]
print(q['quote'], '\n--', q['author'])

{
    "success": {
        "total": 1
    },
    "contents": {
        "quotes": [
            {
                "quote": "If you get up in the morning and think the future is going to be better, it is a bright day. Otherwise, it's not.",
                "length": "113",
                "author": "Elon Musk",
                "tags": [
                    "inspire",
                    "positive-thinking"
                ],
                "category": "inspire",
                "date": "2018-12-05",
                "permalink": "https://theysaidso.com/quote/mbtVO88S_KNdB4gBMyXaUgeF/elon-musk-if-you-get-up-in-the-morning-and-think-the-future-is-going-to-be-bette",
                "title": "Inspiring Quote of the day",
                "background": "https://theysaidso.com/img/bgs/man_on_the_mountain.jpg",
                "id": "mbtVO88S_KNdB4gBMyXaUgeF"
            }
        ],
        "copyright": "2017-19 theysaidso.com"
    }
}
If you get up in the morning and think the future is going

## Cleaning

In [13]:
# Fetch a web page
r = requests.get('https://news.ycombinator.com')
print(r.text[:1000])

<html op="news"><head><meta name="referrer" content="origin"><meta name="viewport" content="width=device-width, initial-scale=1.0"><link rel="stylesheet" type="text/css" href="news.css?PlzieHqljp3FNCn26L0Q">
            <link rel="shortcut icon" href="favicon.ico">
          <link rel="alternate" type="application/rss+xml" title="RSS" href="rss">
        <title>Hacker News</title></head><body><center><table id="hnmain" border="0" cellpadding="0" cellspacing="0" width="85%" bgcolor="#f6f6ef">
        <tr><td bgcolor="#ff6600"><table border="0" cellpadding="0" cellspacing="0" width="100%" style="padding:2px"><tr><td style="width:18px;padding-right:4px"><a href="https://news.ycombinator.com"><img src="y18.gif" width="18" height="18" style="border:1px white solid;"></a></td>
                  <td style="line-height:12pt; height:10px;"><span class="pagetop"><b class="hnname"><a href="news">Hacker News</a></b>
              <a href="newest">new</a> | <a href="newcomments">comments</a> | <a h

In [15]:
# Remove HTML tags using RegEx
pattern = re.compile(r'<.*?>')  # tags look like <...>
print(pattern.sub('', r.text)[:1000])  # replace them with blank


            
          
        Hacker News
        
                  Hacker News
              new | comments | show | ask | jobs | submit            
                              login
                          
              

              
      1.      Why Do Hospitals Hate Sleep So Much? (motherjones.com)
        71 points by curtis 1 hour ago  | hide | 38&nbsp;comments              
      
                
      2.      Announcing Open Source of WPF, Windows Forms, and WinUI (windows.com)
        749 points by MikusR 13 hours ago  | hide | 311&nbsp;comments              
      
                
      3.      The Creepy Line: a documentary about Google, Facebook and user manipulation (thecreepyline.com)
        30 points by znpy 2 hours ago  | hide | 6&nbsp;comments              
      
                
      4.      Facebook ends platform policy banning apps that copy its features (techcrunch.com)
        110 points by brandnewlow 6 hours ago  | hide | 65&nbsp;comments      

In [17]:
# Remove HTML tags using Beautiful Soup library
soup = BeautifulSoup(r.text, 'html5lib')
print(soup.get_text()[:1000])


            
          
        Hacker News
        
                  Hacker News
              new | comments | show | ask | jobs | submit            
                              login
                          
              

              
      1.      Why Do Hospitals Hate Sleep So Much? (motherjones.com)
        71 points by curtis 1 hour ago  | hide | 38 comments              
      
                
      2.      Announcing Open Source of WPF, Windows Forms, and WinUI (windows.com)
        749 points by MikusR 13 hours ago  | hide | 311 comments              
      
                
      3.      The Creepy Line: a documentary about Google, Facebook and user manipulation (thecreepyline.com)
        30 points by znpy 2 hours ago  | hide | 6 comments              
      
                
      4.      Facebook ends platform policy banning apps that copy its features (techcrunch.com)
        110 points by brandnewlow 6 hours ago  | hide | 65 comments              
      
    

In [18]:
# Find all articles
summaries = soup.find_all('tr', class_='athing')
summaries[0]

<tr class="athing" id="18605709">
      <td align="right" class="title" valign="top"><span class="rank">1.</span></td>      <td class="votelinks" valign="top"><center><a href="vote?id=18605709&amp;how=up&amp;goto=news" id="up_18605709"><div class="votearrow" title="upvote"></div></a></center></td><td class="title"><a class="storylink" href="https://www.motherjones.com/kevin-drum/2018/12/why-do-hospitals-hate-sleep-so-much/">Why Do Hospitals Hate Sleep So Much?</a><span class="sitebit comhead"> (<a href="from?site=motherjones.com"><span class="sitestr">motherjones.com</span></a>)</span></td></tr>

In [19]:
# Extract title
summaries[0].find('a', class_='storylink').get_text().strip()

'Why Do Hospitals Hate Sleep So Much?'

In [20]:
# Find all articles, extract titles
articles = []
summaries = soup.find_all('tr', class_='athing')
for summary in summaries:
    title = summary.find('a', class_='storylink').get_text().strip()
    articles.append((title))

print(len(articles), 'Article summaries found. Sample:')
print(articles[0])

30 Article summaries found. Sample:
Why Do Hospitals Hate Sleep So Much?


## Normalization

### Case Normalization

In [26]:
# Sample text
text = ('The first time you see The Second Renaissance it may look boring. Look at it at least '
        'twice and definitely watch part 2. It will change your view of the matrix. Are the '
        'human people the ones who started the war ? Is AI a bad thing ?')
print(text)

The first time you see The Second Renaissance it may look boring. Look at it at least twice and definitely watch part 2. It will change your view of the matrix. Are the human people the ones who started the war ? Is AI a bad thing ?


In [27]:
# Convert to lowercase
text = text.lower() 
print(text)

the first time you see the second renaissance it may look boring. look at it at least twice and definitely watch part 2. it will change your view of the matrix. are the human people the ones who started the war ? is ai a bad thing ?


### Punctuation Removal

In [28]:
# Remove punctuation characters
text = re.sub(r'[^a-zA-Z0-9]', ' ', text) 
print(text)

text = re.sub(r'\W+', ' ', text)
print(text)

the first time you see the second renaissance it may look boring  look at it at least twice and definitely watch part 2  it will change your view of the matrix  are the human people the ones who started the war   is ai a bad thing  
the first time you see the second renaissance it may look boring look at it at least twice and definitely watch part 2 it will change your view of the matrix are the human people the ones who started the war is ai a bad thing 


## Tokenization

In [29]:
# Split text into tokens (words)
words = text.split()
print(words)

['the', 'first', 'time', 'you', 'see', 'the', 'second', 'renaissance', 'it', 'may', 'look', 'boring', 'look', 'at', 'it', 'at', 'least', 'twice', 'and', 'definitely', 'watch', 'part', '2', 'it', 'will', 'change', 'your', 'view', 'of', 'the', 'matrix', 'are', 'the', 'human', 'people', 'the', 'ones', 'who', 'started', 'the', 'war', 'is', 'ai', 'a', 'bad', 'thing']


### NLTK: Natural Language ToolKit

In [31]:
#nltk.data.path.append(os.path.join(os.getcwd(), 'nltk_data'))
nltk.data.path.append(NLTK_DATA)

In [32]:
# Another sample text
text = ('Dr. Smith graduated from the University of Washington. He later started an analytics '
        'firm called Lux, which catered to enterprise customers.')
print(text)

Dr. Smith graduated from the University of Washington. He later started an analytics firm called Lux, which catered to enterprise customers.


In [34]:
# Split text into words using NLTK
words = word_tokenize(text)
print(words)

['Dr.', 'Smith', 'graduated', 'from', 'the', 'University', 'of', 'Washington', '.', 'He', 'later', 'started', 'an', 'analytics', 'firm', 'called', 'Lux', ',', 'which', 'catered', 'to', 'enterprise', 'customers', '.']


In [36]:
# Split text into sentences
sentences = sent_tokenize(text)
print(sentences)

['Dr. Smith graduated from the University of Washington.', 'He later started an analytics firm called Lux, which catered to enterprise customers.']


In [39]:
# List stop words
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [41]:
# Reset text
text = ('The first time you see The Second Renaissance it may look boring. Look at it at least '
        'twice and definitely watch part 2. It will change your view of the matrix. Are the '
        'human people the ones who started the war ? Is AI a bad thing ?')

# Normalize it
text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

# Tokenize it
words = text.split()
print(words)

['the', 'first', 'time', 'you', 'see', 'the', 'second', 'renaissance', 'it', 'may', 'look', 'boring', 'look', 'at', 'it', 'at', 'least', 'twice', 'and', 'definitely', 'watch', 'part', '2', 'it', 'will', 'change', 'your', 'view', 'of', 'the', 'matrix', 'are', 'the', 'human', 'people', 'the', 'ones', 'who', 'started', 'the', 'war', 'is', 'ai', 'a', 'bad', 'thing']


In [42]:
# Remove stop words
words = [w for w in words if w not in stopwords.words('english')]
print(words)

['first', 'time', 'see', 'second', 'renaissance', 'may', 'look', 'boring', 'look', 'least', 'twice', 'definitely', 'watch', 'part', '2', 'change', 'view', 'matrix', 'human', 'people', 'ones', 'started', 'war', 'ai', 'bad', 'thing']


### Sentence Parsing

In [43]:
# Define a custom grammar
my_grammar = nltk.CFG.fromstring('''
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
''')
parser = nltk.ChartParser(my_grammar)

# Parse a sentence
sentence = word_tokenize('I shot an elephant in my pajamas')
for tree in parser.parse(sentence):
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


## Stemming & Lemmatization

### Stemming

In [44]:
# Reduce words to their stems
stemmed = [PorterStemmer().stem(w) for w in words]
print(stemmed)

['first', 'time', 'see', 'second', 'renaiss', 'may', 'look', 'bore', 'look', 'least', 'twice', 'definit', 'watch', 'part', '2', 'chang', 'view', 'matrix', 'human', 'peopl', 'one', 'start', 'war', 'ai', 'bad', 'thing']


### Lemmatization

In [45]:
# Reduce words to their root form
lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
print(lemmed)

['first', 'time', 'see', 'second', 'renaissance', 'may', 'look', 'boring', 'look', 'least', 'twice', 'definitely', 'watch', 'part', '2', 'change', 'view', 'matrix', 'human', 'people', 'one', 'started', 'war', 'ai', 'bad', 'thing']


In [46]:
# Lemmatize verbs by specifying pos
lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
print(lemmed)

['first', 'time', 'see', 'second', 'renaissance', 'may', 'look', 'bore', 'look', 'least', 'twice', 'definitely', 'watch', 'part', '2', 'change', 'view', 'matrix', 'human', 'people', 'one', 'start', 'war', 'ai', 'bad', 'thing']
